In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_15\Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems

In [3]:
function hat(q)
    return [
        0 -q[3] q[2];
        q[3] 0 -q[1];
        -q[2] q[1] 0
    ]
end

function L(q)
    s = q[1]
    v = q[2:4]
    return [
        s -v';
        v s*I+hat(v)
    ]
end

L (generic function with 1 method)

In [4]:
T = Diagonal([1, -1, -1, -1])
H = [ zeros(3)'; I(3) ]

4×3 SparseArrays.SparseMatrixCSC{Float64, Int64} with 3 stored entries:
  ⋅    ⋅    ⋅ 
 1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅ 
  ⋅    ⋅   1.0

In [5]:
function G(q)
    return L(q)*H
end

function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function rptoq(ϕ)
    return (1/sqrt(1+ϕ'*ϕ))*[
        1;
        ϕ
    ]
end
function qtorp(q)
    return q[2:4] / q[1]
end

qtorp (generic function with 1 method)

## Compensation Matrix for Quaternion conversion 

In [6]:
function E(q)
    # multiply 1.0 to resolve BlockDiagonal Err
    return BlockDiagonal([ 1.0*I(3), G(q0), 1.0*I(6) ])
end

E (generic function with 1 method)

## Quadrotor Parameters

In [7]:
#Quadrotor parameters
m = 0.5
ℓ = 0.1750
J = Diagonal([0.0023, 0.0023, 0.004])
g = 9.81
kt=1.0
km=0.0245

h = 0.05 #20 Hz

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

## Quadrotor Dynamics

In [8]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7] / norm(x[4:7])
    v = x[8:10]
    w = x[11:13]
    Q = qtoQ(q)

    ṙ = Q*v
    q̇ = 0.5*L(q)*H*w
    v̇ = Q'*[0;0;-g] + (1/m)*[zeros(2,4); kt kt kt kt]*u - hat(w)*v
    ẇ = J\(-hat(w)*J*w + [0 ℓ*kt 0 -ℓ*kt; -ℓ*kt 0 ℓ*kt 0; km -km km -km]*u)

    return [ ṙ; q̇; v̇; ẇ ]
end

quad_dynamics (generic function with 1 method)

In [9]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
end

quad_dynamics_rk4 (generic function with 1 method)

# **Prepare Simulation**

In [10]:
#Initial Conditions
uhover = (m*g/4)*ones(4)
r0 = [0.0; 0; 1.0]
q0 = [1.0; 0; 0; 0]
v0 = zeros(3)
ω0 = zeros(3)
x0 = [r0; q0; v0; ω0];

# Cost weights
Q = Array(I(Nx̃));
R = Array(.1*I(Nu));

## Linearize dynamics about hover

In [11]:
A = ForwardDiff.jacobian(_x -> quad_dynamics_rk4(_x, uhover), x0)
B = ForwardDiff.jacobian(_u -> quad_dynamics_rk4(x0, _u), uhover);

## Check the rank and controllablity of A 

In [12]:
rank(A)#The naive A matrix will always be rank deficient due to the unit-norm constraint on the quaternion

12

In [13]:
#Controlability Matrix
C = B
for k = 1:(Nx-1)
    C = [C A*C[:,end-(Nu-1):end]]
end
rank(C) #The naive linearized system will always be uncontrollable

12

## Reduced system (Transform Quaternion into RP) & Check the rank and controllability again

In [14]:
# Array parsing required for dlqr input
Ã = Array(E(q0)' * A * E(q0))
B̃ = Array(E(q0)' * B);

In [15]:
#Controlability Matrix
C = B̃
for k = 1:(Nx-1)
    C = [C Ã*C[:,end-(Nu-1):end]]
end
rank(C) #Reduced system is controllable

12

In [16]:
K = dlqr(Ã, B̃, Q, R)

4×12 Matrix{Float64}:
 -0.114603     -4.57809e-14  1.1308   7.39451e-14  …  -0.147816      0.681679
 -9.67035e-14  -0.114603     1.1308   1.39421         -8.87894e-15  -0.681679
  0.114603     -6.63817e-14  1.1308   1.74249e-13      0.147816      0.681679
 -2.35249e-14   0.114603     1.1308  -1.39421          3.70839e-15  -0.681679

In [17]:
#Feedback controller
function controller(x)
    q0 = x0[4:7]
    q = x[4:7]
    ϕ = qtorp(L(q0)'*q)

    Δx = [
        x[1:3] - r0;
        ϕ;
        x[8:10] - v0;
        x[11:13] - ω0
    ]

    u = uhover - K*Δx
end

controller (generic function with 1 method)

## Simulation

In [22]:
xhist = zeros(Nx, Nt)
uhist = zeros(Nu, Nt)
xhist[:,1] = [r0+randn(3); L(q0)*rptoq(0.5*randn(3)); v0; ω0]
for k in 1:(Nt-1)
    uhist[:,k] = controller(xhist[:,k])
    xhist[:,k+1] = quad_dynamics_rk4(xhist[:,k],uhist[:,k])
end

In [23]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8702


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8702))

In [24]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)

false

In [25]:
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)

## Let's try another initial guess

In [26]:
xhist = zeros(Nx, Nt)
uhist = zeros(Nu, Nt)
xhist[:,1] = [r0+randn(3); L(q0)*rptoq(0.7*randn(3)); v0; ω0]
for k in 1:(Nt-1)
    uhist[:,k] = controller(xhist[:,k])
    xhist[:,k+1] = quad_dynamics_rk4(xhist[:,k],uhist[:,k])
end

In [27]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8703


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))

In [28]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)

## LQR works well in even harsh inital guess

In [29]:
xhist = zeros(Nx, Nt)
uhist = zeros(Nu, Nt)
xhist[:,1] = [r0+randn(3); L(q0)*rptoq([1; 0; 0]); v0; ω0]
for k in 1:(Nt-1)
    uhist[:,k] = controller(xhist[:,k])
    xhist[:,k+1] = quad_dynamics_rk4(xhist[:,k],uhist[:,k])
end

In [32]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8706


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8706))

In [33]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)